<a href="https://colab.research.google.com/github/luciaPi/BreakhisNNlecture/blob/main/breakhisNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tento blok je len pre účely priebežného mazania....

In [ ]:
!rm -rf /tmp/adenosis-benign-400x/
!rm -rf /tmp/lobural_malign-400x/
!rm -f adenosis_benign.zip
!rm -f lobural_malign.zip
!rm -rf /tmp/test/
!rm -rf /tmp/train/
!rm -rf /tmp/valid/
!rm -rf /tmp/breakhis/




**Popis datasetu**

K dispozícii sú dva datasety:

Dataset 1 cca 109MB

- 106 benígnych patchov adenosis 400x
Cesta: https://zilinskauniverzita-my.sharepoint.com/:u:/g/personal/cimrak2_uniza_sk/ESNTMX_-R_tOqoOUNfJZZVwBc0kGjpaGj8kLOFJQbzHHQQ?download=1

- 137 malígnych patchov lobural 400x
Cesta: https://zilinskauniverzita-my.sharepoint.com/:u:/g/personal/cimrak2_uniza_sk/EYDAmtsvJ3xCjERDf1iXAMcB_ZPa4s90ZJzn3ynoyG7urA?download=1


Dataset 2 cca 900MB

- 588 benígnych patchov 400x (adenosis, fibroadenoma, phyllodes, tubular)
Cesta: https://zilinskauniverzita-my.sharepoint.com/:u:/g/personal/cimrak2_uniza_sk/EeyTWoKYZaJJrpQPZWh-OoYBlqx0lVYTcopQYvLWg9TR8w?download=1

- 1232 malígnych patchov 400x (ductal, lobural, mucinous, papillary)
Cesta: https://zilinskauniverzita-my.sharepoint.com/:u:/g/personal/cimrak2_uniza_sk/EeKFsy_Azo5Hmehjz3IBWCMBQjS3HpY1E0lzUfHEuDPvFw?download=1


**Stiahnutie datasetu**

In [ ]:
#!wget --no-check-certificate \
#    Cesta \
#    -O /tmp/benign.zip

# Dataset 1
!wget --no-check-certificate \
    https://zilinskauniverzita-my.sharepoint.com/:u:/g/personal/cimrak2_uniza_sk/ESNTMX_-R_tOqoOUNfJZZVwBc0kGjpaGj8kLOFJQbzHHQQ?download=1 \
    -O /tmp/adenosis_benign.zip

!wget --no-check-certificate \
    https://zilinskauniverzita-my.sharepoint.com/:u:/g/personal/cimrak2_uniza_sk/EYDAmtsvJ3xCjERDf1iXAMcB_ZPa4s90ZJzn3ynoyG7urA?download=1 \
    -O /tmp/lobural_malign.zip

# Dataset 2
#!wget --no-check-certificate \
#    https://zilinskauniverzita-my.sharepoint.com/:u:/g/personal/cimrak2_uniza_sk/EeyTWoKYZaJJrpQPZWh-OoYBlqx0lVYTcopQYvLWg9TR8w?download=1 \
#    -O /tmp/adenosis_benign.zip

#!wget --no-check-certificate \
#    https://zilinskauniverzita-my.sharepoint.com/:u:/g/personal/cimrak2_uniza_sk/EeKFsy_Azo5Hmehjz3IBWCMBQjS3HpY1E0lzUfHEuDPvFw?download=1 \
#    -O /tmp/lobural_malign.zip



**Rozpackovanie datasetu**

In [ ]:
import os
import zipfile

local_zip = '/tmp/adenosis_benign.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
!rm -rf /tmp/adenosis_benign.zip

local_zip = '/tmp/lobural_malign.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()
!rm -rf /tmp/lobural_malign.zip

**Rozdelenie na trénovaciu, validačnú a testovaciu sadu**

In [ ]:
import os
import random
import shutil

# Dataset 1
data_benign_path = "/tmp/adenosis-benign-400x/"
data_malignant_path = "/tmp/lobural_malign-400x/"

# Dataset 2
#data_benign_path = "/tmp/benign-400x/"
#data_malignant_path = "/tmp/malignant-400x/"

# path to destination folders
train_benign_dir = "/tmp/breakhis/train/benign"
valid_benign_dir = "/tmp/breakhis/valid/benign"
test_benign_dir = "/tmp/breakhis/test/benign"
train_malignant_dir = "/tmp/breakhis/train/malignant"
valid_malignant_dir = "/tmp/breakhis/valid/malignant"
test_malignant_dir = "/tmp/breakhis/test/malignant"

# Define a list of image extensions
image_extensions = ['.png']

# Create a list of image filenames in 'data_path'
imgs_list_benign = [filename for filename in os.listdir(data_benign_path) if os.path.splitext(filename)[-1] in image_extensions]
imgs_list_malignant = [filename for filename in os.listdir(data_malignant_path) if os.path.splitext(filename)[-1] in image_extensions]

print('pocet v imgs_list_benign ',len(imgs_list_benign))
print('pocet v imgs_list_malignant ',len(imgs_list_malignant))


**Rozdelenie datasetu**

*   Trénovacie dáta - tréning modelu
*   Validačné dáta - tuning parametrov modelu
*   Testovacie dáta - záverečné testovanie modelu

In [ ]:
# Sets the random seed
random.seed(42)

# Shuffle the list of image filenames
random.shuffle(imgs_list_benign)
random.shuffle(imgs_list_malignant)

# determine the number of images for each set
train_size_benign = int(len(imgs_list_benign) * 0.70)
valid_size_benign = int(len(imgs_list_benign) * 0.15)
test_size_benign = int(len(imgs_list_benign) * 0.15)
train_size_malignant = int(len(imgs_list_malignant) * 0.70)
valid_size_malignant = int(len(imgs_list_malignant) * 0.15)
test_size_malignant = int(len(imgs_list_malignant) * 0.15)
print('train_size_benign ', train_size_benign)
print('valid_size_benign ', valid_size_benign)
print('test_size_benign ', test_size_benign)
print('train_size_malignant ', train_size_malignant)
print('valid_size_malignant ', valid_size_malignant)
print('test_size_malignant ', test_size_malignant)


In [ ]:
# Create destination folders if they don't exist
for folder_path in [train_benign_dir, valid_benign_dir, test_benign_dir]:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
for folder_path in [train_malignant_dir, valid_malignant_dir, test_malignant_dir]:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

# Copy image files to destination folders
for i, f in enumerate(imgs_list_benign):
    if i < train_size_benign:
        dest_folder = train_benign_dir
    elif i < train_size_benign + valid_size_benign:
        dest_folder = valid_benign_dir
    else:
        dest_folder = test_benign_dir
    shutil.copy(os.path.join(data_benign_path, f), os.path.join(dest_folder, f))

for i, f in enumerate(imgs_list_malignant):
    if i < train_size_malignant:
        dest_folder = train_malignant_dir
    elif i < train_size_malignant + valid_size_malignant:
        dest_folder = valid_malignant_dir
    else:
        dest_folder = test_malignant_dir
    shutil.copy(os.path.join(data_malignant_path, f), os.path.join(dest_folder, f))

# Dataset 1
!rm -rf /tmp/adenosis-benign-400x/
!rm -rf /tmp/lobural_malign-400x/

# Dataset 2
!rm -rf /tmp/benign-400x/
!rm -rf /tmp/malignant-400x/


In [ ]:
train_benign_names = os.listdir(train_benign_dir)
print(train_benign_names[:10])

train_malignant_names = os.listdir(train_malignant_dir)
print(train_malignant_names[:10])

validation_benign_names = os.listdir(valid_benign_dir)
print(validation_benign_names[:10])

validation_malignant_names = os.listdir(valid_malignant_dir)
print(validation_malignant_names[:10])

In [ ]:
print('total training benign images:', len(os.listdir(train_benign_dir)))
print('total validation benign images:', len(os.listdir(valid_benign_dir)))
print('total test benign images:', len(os.listdir(test_benign_dir)))
print('total training malignant images:', len(os.listdir(train_malignant_dir)))
print('total validation malignant images:', len(os.listdir(valid_malignant_dir)))
print('total test malignant images:', len(os.listdir(test_malignant_dir)))

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 2
ncols = 4

# Index for iterating over images
pic_index = 0

# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_benign_pic = [os.path.join(train_benign_dir, fname)
                for fname in train_benign_names[pic_index-8:pic_index]]

for i, img_path in enumerate(next_benign_pic):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [ ]:
# Index for iterating over images
pic_index = 0

# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_malignant_pic = [os.path.join(train_malignant_dir, fname)
                for fname in train_malignant_names[pic_index-8:pic_index]]

for i, img_path in enumerate(next_malignant_pic):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 120 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/tmp/breakhis/train/',  # This is the source directory for training images
        classes = ['benign', 'malignant'],
        target_size=(224, 224),  # All images will be resized to 224x224
        batch_size = 20,
        # Use binary labels
        class_mode='binary')

# Flow validation images in batches of 19 using valid_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        '/tmp/breakhis/valid/',  # This is the source directory for training images
        classes = ['benign', 'malignant'],
        target_size=(224, 224),  # All images will be resized to 200x200
        batch_size = 10,
        # Use binary labels
        class_mode='binary',
        shuffle=False)

# Flow test images in batches of 19 using test_datagen generator
test_generator = test_datagen.flow_from_directory(
        '/tmp/breakhis/test/',  # This is the source directory for training images
        classes = ['benign', 'malignant'],
        target_size=(224, 224),  # All images will be resized to 200x200
        batch_size = 10,
        # Use binary labels
        class_mode='binary',
        shuffle=False)

In [ ]:
import tensorflow as tf
import numpy as np
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

**Vytvorenie modelu**

<img src="https://drive.google.com/uc?id=1rhtONHuZP2tF6ECEJtAzMy4lmAkozZ42" alt="Dataset" width=90%>

Vrstvy neurónovej siete:
*   Flatten - z 2D do 1D
*   Dense - klasická vrstva

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape = (224,224,3)),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = tf.optimizers.Adam(learning_rate=0.001),
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator,
      steps_per_epoch = 8,
      epochs = 15,
      verbose = 1,
      validation_data = validation_generator,
      validation_steps = 8)

In [ ]:
model.evaluate(validation_generator)

In [ ]:
model.evaluate(test_generator)

**Úloha - zlepšite model**
- Zmeňte architektúru siete - pridajte vrstvu, počet neurónov
- Zmeňte learning rate


**Convolutional neural network**

<img src="https://drive.google.com/uc?id=1gGigTVgYztXnfsGrycq5dWEgthwpZNr1" alt="CNN1" width=90%>

<img src="https://drive.google.com/uc?id=1WFKxXOSEWPWajwpywWrg2oQh9OD26ZoL" alt="CNN2" width=90%>